# Tarea 2: Uso de técnicas de PLN para representar texto

**Ricardo Calvo Perez - A01028889**

### 0. Importar librerias

In [15]:
!pip install gensim
!pip install nrclex
!wget -nc https://raw.githubusercontent.com/aditeyabaral/lok-sabha-election-twitter-analysis/master/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt -P /content/

from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler
from nltk.corpus import sentiwordnet as swn
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import collections
import spacy
import nltk
import re
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from nrclex import NRCLex
from gensim.models import Word2Vec

nltk.download('stopwords')


File ‘/content/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt’ already there; not retrieving.



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
labels = []
raw_X = []

with open('/content/originallTraining.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 3:
            labels.append(parts[2])
            raw_X.append('\t'.join(parts[3:]))

for line in raw_X[:10]:
    print(line)

unique_labels = list(set(labels))
print("\nUnique labels:")
print(unique_labels)

Gas by my house hit $3.39!!!! I'm going to Chapel Hill on Sat. :)
Iranian general says Israel's Iron Dome can't deal with their missiles (keep talking like that and we may end up finding out)
with J Davlar 11th. Main rivals are team Poland. Hopefully we an make it a successful end to a tough week of training tomorrow.
Talking about ACT's && SAT's, deciding where I want to go to college, applying to colleges and everything about college stresses me out.
They may have a SuperBowl in Dallas, but Dallas ain't winning a SuperBowl. Not with that quarterback and owner. @S4NYC @RasmussenPoll
Im bringing the monster load of candy tomorrow, I just hope it doesn't get all squiched
Apple software, retail chiefs out in overhaul: SAN FRANCISCO Apple Inc CEO Tim Cook on Monday replaced the heads... http://bit.ly/XQEhJU
#Livewire Nadal confirmed for Mexican Open in February: Rafael Nadal is set to play at the Me... http://bit.ly/WY4Vjy  #LiveWireAthletics
#Iran US delisting MKO from global terrorists 

In [21]:
from tqdm.auto import trange
import re, spacy
from nltk.corpus import stopwords

def clean_text(raw_x, leave_caps = True, custom_stopwords = None, possible_emojis = None, intesifiers = None, negators = None, emoji_map = None, return_all = False):

  processed_caps_X, processed_regex_X, processed_stopwords_X = [], [], []
  processed_special_characters_X, processed_POS_tags_X, processed_emojis_X = [], [], []

  # Step 1: Process caps
  # This loop processes each line in raw_X, converting words to lowercase
  # except when the word is fully uppercase and leave_caps is True.
  for i in trange(len(raw_x), desc="Step 1: Process caps"):
    line = raw_x[i]
    words = line.split()  # Split the line into individual words
    processed_line = []   # Temporary list to hold processed words

    for word in words:
        # If the word is fully uppercase and we want to preserve caps, keep it as-is
        if word.isupper() and leave_caps:
            processed_line.append(word)
        else:
            # Otherwise, convert the word to lowercase
            processed_line.append(word.lower())

    # Join processed words back into a single string and add to processed_caps_X
    processed_caps_X.append(" ".join(processed_line))

  # Step 2: Process URLs and mentions
  # This loop cleans each line in processed_caps_X by removing URLs and @mentions.
  for i in trange(len(processed_caps_X), desc="Step 2: Process URLs & mentions"):
      line = processed_caps_X[i]
      # Remove URLs starting with "http" followed by any non-space characters
      line = re.sub(r'http\S+', '', line)
      # Remove URLs starting with "www" followed by any characters
      line = re.sub(r'www.+', '', line)

      # Remove mentions that start with '@' followed by non-space characters
      line = re.sub(r'@\S+', '', line)

      # Append the cleaned line to processed_regex_X
      processed_regex_X.append(line)

  # Step 3: Process stop-words
  # This step removes common stopwords from the text, but keeps certain useful words like "not" or "very".
  if custom_stopwords is None:
      # Load default English stopwords from NLTK
      stopwords_en = set(stopwords.words("english"))
      # Words to keep even though they appear in standard stopword lists
      keep_words = {
          "no","not","never","none","nobody","nothing","neither","nor","nowhere",
          "hardly","barely","scarcely","very","too","so","quite","really","super","won"
      }

      # Final stopword set: all stopwords except those in keep_words
      custom_stopwords = stopwords_en - keep_words
  else:
      # Ensure the provided stopwords are a set for faster lookup
      custom_stopwords = set(custom_stopwords)

  # Loop through each cleaned line and remove stopwords
  for i in trange(len(processed_regex_X), desc="Step 3: Process stop-words"):
      line = processed_regex_X[i]
      tokens = line.split()      # split line into words
      kept_tokens = []           # temporary list to store non-stopwords

      for t in tokens:
          # Compare lowercase version to handle capitalization differences
          if t.lower() not in custom_stopwords:
              kept_tokens.append(t)                # keep non-stopword tokens

      # Join remaining words back into a cleaned line
      processed_stopwords_X.append(" ".join(kept_tokens))

  # Step 4: Process special characters
  # This step normalizes punctuation and emojis, removes unwanted symbols,
  # and ensures text remains clean and readable for further processing.
  if possible_emojis is None:
      # Default set of common emoticons to protect before cleaning
      possible_emojis = sorted({
          ":-)", ":-D", ":-(", ";-)", ":-/", ":/", ":)", "(:", ":D", ":(", "):", "(;", ";)",
          ":P", ":-P", "<3", ":'(", ":'-(", ":|", ":-|", ":o", ":O", ":-o", ":-O", ":*", ":-*",
          ":p", ":-p", ";p", ";P", "xD", "XD", "xDD", "XDD", "T_T", "TT_TT", "u_u", "U_U",
          "-_-", "^_^", "^-^", "o.o", "O_O", "x_x", "X_X", "~_~"
      }, key=len, reverse=True)
  else:
      # If a custom emoji list is given, sort by length for proper matching
      possible_emojis = sorted(possible_emojis, key=len, reverse=True)

  for i in trange(len(processed_stopwords_X), desc="Step 4: Process special chars"):
      line = processed_stopwords_X[i]
      # 0) Normalize smart quotes and other variants for consistency
      line = (line.replace("’", "'").replace("‘", "'").replace("`", "'")
                  .replace("“", '"').replace("”", '"'))

      # 1) Temporarily replace emoticons with markers to prevent deletion
      protected = {}
      for j, emo in enumerate(possible_emojis):
          if emo in line:
              marker = f"EMO_{j}_TOKEN"
              protected[marker] = emo
              line = line.replace(emo, marker)

      # 2) Remove commas used as thousand separators (e.g., "10,000" → "10000")
      line = re.sub(r'(?<=\d),(?=\d)', '', line)

      # 3) Replace exclamation and question sequences with a space to prevent word merging
      line = re.sub(r'[!?]+', ' ', line)

      # 4) Replace quotes and dashes with spaces to separate joined words
      line = re.sub(r'["“”]+', ' ', line)
      line = re.sub(r'[–—]+', ' ', line)

      # 5) Remove all non-word, non-space, non-apostrophe characters
      line = re.sub(r"[^\w\s']+", ' ', line)

      # 6) Remove stray apostrophes not between letters
      line = re.sub(r"(?<![A-Za-z])'(?![A-Za-z])", ' ', line)

      # 7) Collapse multiple spaces into one and trim edges
      line = re.sub(r'\s+', ' ', line).strip()

      # 8) Restore emoticons from protected markers
      for marker, emo in protected.items():
          line = line.replace(marker, emo)

      # Add the fully cleaned line to the final list
      processed_special_characters_X.append(line)

  # Step 6: Process PoS Tags
  # Step 7: Process Lemmatization
  # Load the small English model with NER disabled for speed.
  nlp_en = spacy.load("en_core_web_sm", disable=["ner"])
  # Prepare intensifiers
  if intesifiers is None:
      # Default set of English/Spanish intensifiers
      intesifiers = {
          "very","really","so","too","quite","super","extremely","re",
          "muy","súper","tan","bastante"
      }
  else:
      # Ensure fast membership checks
      intensifiers = set(intensifiers)

  # Prepare negators
  if negators is None:
      # Default set of English/Spanish negators
      negators = {
          "no","not","never","none","nobody","nothing","neither","nor","nowhere",
          "hardly","barely","scarcely",
          "nunca","jamás","ni","tampoco","sin"
      }
  else:
      # Ensure fast membership checks
      negators = set(negators)

  for i in trange(len(processed_special_characters_X), desc="Step 6–7: Process POS & Lemma"):
      line = processed_special_characters_X[i]
      # Process the tweet with spaCy (tokenization, lemma, PoS info)
      doc = nlp_en(line)
      out = []
      i_tok = 0
      while i_tok < len(doc):
          tok = doc[i_tok]
          low = tok.text.lower()

          # --- 1.6 PoS Tags (Negation & Intensifiers) ---
          # If token is a negator or intensifier and has a next token,
          # merge them into a single token like "not_good" or "very_happy".
          if (low in negators or low in intesifiers) and i_tok+1 < len(doc):
              next_token = doc[i_tok+1].lemma_.lower()  # use lemma of next word
              out.append(f"{low}_{next_token}")
              i_tok += 2
              continue

          # --- 1.7 Lemmatization ---
          # For normal tokens, append the lemma in lowercase
          # (e.g., "running" → "run").
          lemma = tok.lemma_.lower()

          if low == "'s":   # it's, he's, she's, that's
              lemma = "is"
          elif low == "'m":  # I'm
              lemma = "am"
          elif low == "'re": # they're, you're, we're
              lemma = "are"
          elif low == "'ll": # I'll, he'll, she'll, they'll
              lemma = "will"
          elif low == "'ve": # I've, they've, we've, you've
              lemma = "have"
          elif low == "'d":  # I'd, he'd, she'd, they'd
              # puede ser "would" o "had", contexto dependiente → en análisis simple usamos "would"
              lemma = "would"

          out.append(lemma)
          i_tok += 1

      processed_POS_tags_X.append(" ".join(out))

  # Step 8: Process emojis
  if emoji_map is None:
      emoji_map = {
    # Positive
    ":-)": "happy", ":)": "happy", "(:": "happy",
    ":D": "smile", ":-D": "smile", "xD": "smile", "XD": "smile", "xDD": "smile", "XDD": "smile",
    "<3": "love",

    # Negative
    ":-(": "sad", ":(": "sad", "):": "sad",
    ":'(": "cry", ":'-(": "cry",
    "T_T": "cry", "TT_TT": "cry",
    "x_x": "dead", "X_X": "dead",
    "-_-": "annoyed",

    # Neutral / uncertain / surprised
    ":-/": "uncertain", ":/": "uncertain",
    ":|": "neutral", ":-|": "neutral",
    ":o": "surprised", ":O": "surprised",
    ":-o": "surprised", ":-O": "surprised",
    "o.o": "surprised", "O_O": "surprised",

    # Playful / wink
    ";-)": "wink", ";)": "wink", "(;": "wink",
    ":P": "playful", ":-P": "playful", ";P": "playful", ";p": "playful",
    ":p": "playful", ":-p": "playful",

    # Cute / positive
    "^_^": "smile", "^-^": "smile",
    "u_u": "sad", "U_U": "sad",
    "~_~": "tired",
    ":*": "kiss", ":-*": "kiss",
}

  for i in trange(len(processed_POS_tags_X), desc="Step 8: Process emojis"):
        line = processed_POS_tags_X[i]
        for emo, token in emoji_map.items():
            if emo in line:
                line = line.replace(emo, token)
        processed_emojis_X.append(line)

  if return_all:
    return processed_caps_X, processed_regex_X, processed_stopwords_X, processed_special_characters_X, processed_POS_tags_X, processed_emojis_X

  return processed_emojis_X


In [22]:
clean_X = clean_text(raw_X)

for text in clean_X[:10]:
    print(text)

Step 1: Process caps:   0%|          | 0/6165 [00:00<?, ?it/s]

Step 2: Process URLs & mentions:   0%|          | 0/6165 [00:00<?, ?it/s]

Step 3: Process stop-words:   0%|          | 0/6165 [00:00<?, ?it/s]

Step 4: Process special chars:   0%|          | 0/6165 [00:00<?, ?it/s]

Step 6–7: Process POS & Lemma:   0%|          | 0/6165 [00:00<?, ?it/s]

Step 8: Process emojis:   0%|          | 0/6165 [00:00<?, ?it/s]

gas house hit 3 39 go chapel hill sit happy
iranian general say israel is iron dome can not deal missile keep talk like may end find out
j davlar 11th main rival team poland hopefully make successful end tough week training tomorrow
talk act is sit is decide want go college apply college everything college stress out
may superbowl dallas dallas be not win superbowl not_quarterback owner
i m bring monster load candy tomorrow hope get squiche
apple software retail chief overhaul san francisco apple inc ceo tim cook monday replace head
livewire nadal confirm mexican open february rafael nadal set play i livewireathletic
iran us delist mko global terrorist list line iran campaign tehran oct 30 irna secretary
expect light moderate rain e visayas cebu bohol samar leyte 30 70 chance rain tonight expect fair weather tomorrow happy
